自然语言处理应用可以划分成几个类别：
- 序列级
    - 单文本分类：情绪分析*
    - 文本对分类：自然语言推断
- 词元级
    - 文本标注
    - 问答任务

在单文本分类应用中，特殊分类标记“<cls>”的BERT表示对整个输入文本序列的信息进行编码。作为输入单个文本的表示，它将被送入到由全连接（稠密）层组成的小多层感知机中，以输出所有离散标签值的分布。


《D2L》使用前一节定义的数据集做自然语言推断，微调 BERT

## 1.加载预训练 BERT

In [1]:
import json
import multiprocessing
import os
import torch
from torch import nn
from d2l import torch as d2l

两个版本的预训练的BERT：
“bert.base”与原始的BERT基础模型一样大，需要大量的计算资源才能进行微调，
而“bert.small”是一个小版本，以便于演示。

In [2]:
# d2l.DATA_HUB['bert.base'] = (d2l.DATA_URL + 'bert.base.torch.zip',
#                              '225d66f04cae318b841a13d32af3acc165f253ac')
d2l.DATA_HUB['bert.small'] = (d2l.DATA_URL + 'bert.small.torch.zip',
                              'c72329e68a732bef0452e4b96a1c341c8910f81f')

预训练好的BERT模型包含
- 定义词表的 `vocab.json`
- 预训练参数的 `pretrained.params`
接下来加载这些参数

In [3]:
def load_pretrained_model(pretrained_model, num_hiddens, ffn_num_hiddens,
                          num_heads, num_layers, dropout, max_len, devices):
    data_dir = d2l.download_extract(pretrained_model)
    # 定义空词表以加载预定义词表
    vocab = d2l.Vocab()
    vocab.idx_to_token = json.load(open(os.path.join(data_dir,
        'vocab.json')))
    vocab.token_to_idx = {token: idx for idx, token in enumerate(
        vocab.idx_to_token)}

    bert = d2l.BERTModel(len(vocab), num_hiddens, norm_shape=[256],
                         ffn_num_input=256, ffn_num_hiddens=ffn_num_hiddens,
                         num_heads=4, num_layers=2, dropout=0.2,
                         max_len=max_len, key_size=256, query_size=256,
                         value_size=256, hid_in_features=256,
                         mlm_in_features=256, nsp_in_features=256)
    # 加载预训练BERT参数
    bert.load_state_dict(torch.load(os.path.join(data_dir,
                                                 'pretrained.params')))
    return bert, vocab

In [4]:
devices = d2l.try_all_gpus()
bert, vocab = load_pretrained_model(
    'bert.small', num_hiddens=256, ffn_num_hiddens=512, num_heads=4,
    num_layers=2, dropout=0.1, max_len=512, devices=devices)

## 2. 微调 BERT 数据集

对于 SNLI 数据集的下游任务自然语言推断，我们定义了一个定制的数据集类 SNLIBERTDataset。
在每个样本中，前提和假设形成一对文本序列，并被打包成一个BERT input vector
segment embedding 用于区分 BERT 输入序列中的前提和假设。
利用预定义的BERT输入序列的最大长度（max_len），持续移除 输入文本对 中较长文本的最后一个标记，直到满足max_len。
为了加速生成用于微调 BERT 的 SNLI 数据集，开 4 个工作进程并行生成 训练/测试样本。

In [5]:
class SNLIBERTDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, max_len, vocab=None):
        # all_premise_hypothesis_tokens[i][0] 表示第 i 个句子对的“前提句”（该句是一个 list(int)）,以此类推
        all_premise_hypothesis_tokens = [
            [p_tokens, h_tokens] for p_tokens, h_tokens in zip(
            *[d2l.tokenize([s.lower() for s in sentences])
              for sentences in dataset[:2]])]

        self.labels = torch.tensor(dataset[2])
        self.vocab = vocab
        self.max_len = max_len
        (self.all_token_ids, self.all_segments,
         self.valid_lens) = self._preprocess(all_premise_hypothesis_tokens)
        print('read ' + str(len(self.all_token_ids)) + ' examples')

    #Python 是单线程的，数据预处理非常慢，要么开多线程，要么用 C++
    def _preprocess(self, all_premise_hypothesis_tokens):
        pool = multiprocessing.Pool(4)  # 使用4个进程
        out = pool.map(self._mp_worker, all_premise_hypothesis_tokens) # map 将函数_mp_worker() 挂载到每个元素上
        # 具体来说，all_premise_hypothesis_tokens 中的每个元素都传参到 _mp_worker() 一次，这项工作由4个进程分别同时完成

        all_token_ids = [
            token_ids for token_ids, segments, valid_len in out]
        all_segments = [segments for token_ids, segments, valid_len in out]
        valid_lens = [valid_len for token_ids, segments, valid_len in out]

        return (torch.tensor(all_token_ids, dtype=torch.long),
                torch.tensor(all_segments, dtype=torch.long),
                torch.tensor(valid_lens))

    def _mp_worker(self, premise_hypothesis_tokens):
        p_tokens, h_tokens = premise_hypothesis_tokens
        self._truncate_pair_of_tokens(p_tokens, h_tokens)
        tokens, segments = d2l.get_tokens_and_segments(p_tokens, h_tokens)

        # 完成填充数据的任务
        token_ids = self.vocab[tokens] + [self.vocab['<pad>']] * (self.max_len - len(tokens))
        segments = segments + [0] * (self.max_len - len(segments))
        valid_len = len(tokens)
        return token_ids, segments, valid_len

    def _truncate_pair_of_tokens(self, p_tokens, h_tokens):
        # 为BERT输入中的'<CLS>'、'<SEP>'和'<SEP>'词元保留位置
        while len(p_tokens) + len(h_tokens) > self.max_len - 3:
            if len(p_tokens) > len(h_tokens):
                p_tokens.pop()
            else:
                h_tokens.pop()

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx]), self.labels[idx]

    def __len__(self):
        return len(self.all_token_ids)

下载完SNLI数据集后，我们通过实例化SNLIBERTDataset类来生成训练和测试样本。
这些样本将在自然语言推断的训练和测试期间进行小批量读取。

In [ ]:
# 如果出现显存不足错误，请减少“batch_size”。在原始的BERT模型中，max_len=512
batch_size, max_len, num_workers = 16, 128, d2l.get_dataloader_workers()
# data_dir = d2l.download_extract('SNLI')
data_dir = "../data/snli_1.0"
train_set = SNLIBERTDataset(d2l.read_snli(data_dir, True), max_len, vocab)
test_set = SNLIBERTDataset(d2l.read_snli(data_dir, False), max_len, vocab)

train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True,num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(test_set, batch_size,num_workers=num_workers)

## 3. 微调 BERT
用于自然语言推断的微调BERT只需要一个由两个全连接层组成的MLP。
这个多层感知机将特殊的“<cls>”词元的 BERT 表示进行了转换，该词元同时编码前提和假设的信息为自然语言推断的三个输出：蕴涵、矛盾和中性。

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, bert):
        super(BERTClassifier, self).__init__()
        self.encoder = bert.encoder
        self.hidden = bert.hidden
        self.output = nn.Linear(256, 3)

    def forward(self, inputs):
        tokens_X, segments_X, valid_lens_x = inputs
        encoded_X = self.encoder(tokens_X, segments_X, valid_lens_x)
        # 保留(batch_size,句首的<cls>,向量长度)
        return self.output(self.hidden(encoded_X[:, 0, :]))

预训练的BERT模型 bert 被送到用于下游应用的BERTClassifier实例net中。
在BERT微调的常见实现中，只有额外的多层感知机（net.output）的输出层的参数将从零开始学习。
预训练BERT编码器（net.encoder）和额外的多层感知机的隐藏层（net.hidden）的所有参数都将进行微调。

In [ ]:
net = BERTClassifier(bert)

预训练的 BERT 中不是所有的参数都会被微调到。

MLM 类和 NSP 类在其使用的多层感知机中都有一些参数。这些参数是预训练BERT模型 bert 中参数的一部分，因此也是net中的参数的一部分。
然而，这些参数仅用于计算预训练过程中的遮蔽语言模型损失和下一句预测损失。
这两个损失函数与微调下游应用无关，因此当BERT微调时，MaskLM和NextSentencePred中采用的多层感知机的参数不会更新（陈旧的，staled）。

为了允许具有陈旧梯度的参数，标志 `ignore_stale_grad=True` 在step函数 `d2l.train_batch_ch13` 中被设置。
我们通过该函数使用SNLI的训练集（train_iter）和测试集（test_iter）对net模型进行训练和评估。

In [ ]:
lr, num_epochs = 1e-4, 5
trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction='none')
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs,devices)

## QA
1. 请问Bert微调的时候，是固定预训练模型的参数吗？
> 一般不，所有的参数跟着重新训练。但可以试一下，可以加速训练。

2. PC 的性能，想跑 BERT 怎么办？
> 用模型蒸馏技术把参数降低到原来的1/10